In [1]:
import numpy as np
import pandas as pd
import os
import re
from glob import glob

In [2]:
G34_raw_data = sorted(glob('./raw_data/G34/G*.txt'))

In [3]:
G34_raw_data_name = list(map(lambda x:x[15:],G34_raw_data))

In [4]:
i = 0
all_data = pd.DataFrame()
for file in G34_raw_data:
    data = pd.read_csv(file, sep = '\t')
    data['data_name'] = G34_raw_data_name[i]
    data.columns = data.columns.str.replace('  ','')
    data.columns = data.columns.str.replace('G34 -','G34-') #有些奇怪的間隔
    data['data_point'] = data.index
    all_data = pd.concat([data,all_data])
    all_data = all_data.drop([0]) #消除單位Deg.
    all_data.reset_index(drop = True)
    i = i + 1

C:\Users\AutoUser\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [5]:
all_data.columns

Index(['G34-PTC1', 'G34-PTC10', 'G34-PTC11', 'G34-PTC12', 'G34-PTC13',
       'G34-PTC14', 'G34-PTC15', 'G34-PTC16', 'G34-PTC17', 'G34-PTC18',
       'G34-PTC19', 'G34-PTC2', 'G34-PTC20', 'G34-PTC21', 'G34-PTC22',
       'G34-PTC23', 'G34-PTC24', 'G34-PTC25', 'G34-PTC26', 'G34-PTC28',
       'G34-PTC29', 'G34-PTC3', 'G34-PTC30', 'G34-PTC31', 'G34-PTC32',
       'G34-PTC33', 'G34-PTC34', 'G34-PTC35', 'G34-PTC36', 'G34-PTC37',
       'G34-PTC38', 'G34-PTC39', 'G34-PTC4', 'G34-PTC40', 'G34-PTC41',
       'G34-PTC42', 'G34-PTC43', 'G34-PTC44', 'G34-PTC45', 'G34-PTC46',
       'G34-PTC47', 'G34-PTC48', 'G34-PTC5', 'G34-PTC6', 'G34-PTC7',
       'G34-PTC8', 'G34-PTC9', 'Unnamed: 8', 'data_name', 'data_point'],
      dtype='object')

In [6]:
#drop 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8' 的columns
#rows出現 Deg的資料 欄位
for column in all_data.columns:
    if column[0] == 'U':
        all_data.drop([column],axis = 1,inplace = True)

In [7]:
all_data['data_name'] = all_data['data_name'].apply(lambda x:x[:-11]) #去掉 _Export.txt

In [8]:
all_data['data_name'].tail()

377    G34 -AC(12X30)20170506-001
378    G34 -AC(12X30)20170506-001
379    G34 -AC(12X30)20170506-001
380    G34 -AC(12X30)20170506-001
381    G34 -AC(12X30)20170506-001
Name: data_name, dtype: object

In [9]:
all_data['recipes'] = all_data['data_name'].str.extract(r'(G\d+)',expand = False) #抓出所屬的Recipes
all_data['autoclave'] = all_data['data_name'].str.extract(r'(AC\(\d+x\d+\))',expand = False,flags=re.IGNORECASE) #用正則抓出 所屬的熱壓爐
all_data['date'] = all_data['data_name'].str.extract(r'(\d{7,8})',expand = False) #抓出日期
all_data['number'] = all_data['data_name'].str.extract(r'(00\d{1,3})',expand = False)#抓出所屬編號
all_data.drop(['data_name'],axis = 1,inplace = True)

In [10]:
all_data.columns = all_data.columns.str.replace('G34-','')

In [11]:
PTC_list = []
for column in all_data.columns:
    if column[0] == 'P' and column[-1] != 't':
        PTC_list.append(column)

In [12]:
all_data['PTC_count'] = len(PTC_list) - (all_data.shape[1] - all_data.count(axis = 1)) 
#每筆資料所收集溫度參數的個數

In [13]:
all_data.columns

Index(['PTC1', 'PTC10', 'PTC11', 'PTC12', 'PTC13', 'PTC14', 'PTC15', 'PTC16',
       'PTC17', 'PTC18', 'PTC19', 'PTC2', 'PTC20', 'PTC21', 'PTC22', 'PTC23',
       'PTC24', 'PTC25', 'PTC26', 'PTC28', 'PTC29', 'PTC3', 'PTC30', 'PTC31',
       'PTC32', 'PTC33', 'PTC34', 'PTC35', 'PTC36', 'PTC37', 'PTC38', 'PTC39',
       'PTC4', 'PTC40', 'PTC41', 'PTC42', 'PTC43', 'PTC44', 'PTC45', 'PTC46',
       'PTC47', 'PTC48', 'PTC5', 'PTC6', 'PTC7', 'PTC8', 'PTC9', 'data_point',
       'recipes', 'autoclave', 'date', 'number', 'PTC_count'],
      dtype='object')

In [14]:
all_data['TemParameter'] = 0

In [15]:
G34_data = pd.DataFrame()

In [16]:
for PTC in PTC_list:
    data = all_data.loc[~all_data[PTC].isnull(),
             ['number', 'date', 'autoclave', 'recipes'
              ,'TemParameter', 'PTC_count','data_point',PTC]] #將各別 PTC 非0的數據挑出
    data['TemParameter'] = PTC  #建立 TemParameter 對應所屬來的PTC
    data['value'] = data[PTC]  # 賦予value值 統一
    data.drop([PTC],inplace=True,axis = 1)
    G34_data = pd.concat([G34_data,data])

In [17]:
G34_data.reset_index(drop=True, inplace=True)

In [18]:
G34_data['TemParameter'].value_counts()

PTC1     4582
PTC32    3834
PTC22    3552
PTC6     3337
PTC44    3079
PTC34    3070
PTC9     3069
PTC7     3067
PTC14    2991
PTC3     2966
PTC16    2738
PTC21    2691
PTC41    2689
PTC25    2683
PTC17    2679
PTC42    2634
PTC40    2628
PTC29    2611
PTC35    2318
PTC5     2317
PTC37    2278
PTC45    2249
PTC26    2229
PTC20    2225
PTC8     1992
PTC15    1944
PTC30    1914
PTC33    1905
PTC13    1885
PTC36    1868
PTC28    1867
PTC43    1844
PTC31    1549
PTC2     1539
PTC38    1518
PTC4     1517
PTC18    1503
PTC12    1501
PTC11    1491
PTC46    1176
PTC48    1138
PTC39     768
PTC10     761
PTC47     747
PTC19     738
PTC23     387
PTC24     380
Name: TemParameter, dtype: int64

In [19]:
G34_data.to_csv('./training_data/G34_data.csv')